In [162]:
import pandas as pd
from numba import jit
from biopandas.pdb import PandasPdb
import numpy as np
import csv
import time
import os
from antibody.SequenceUtils import *


## Adding Meiler's feature for padding along with one hot encoding based on structural properties

In [163]:
def seq_to_one_hot(res_seq_one):
    from keras.utils.np_utils import to_categorical
    ints = one_to_number(res_seq_one)
    feats = aa_features()[ints]
    onehot = to_categorical(ints, num_classes=len(aa_s))
    return np.concatenate((onehot, feats), axis=1)

aa_s = "CSTPAGNDEQHRKMILVFYWX"
def one_to_number(res_str):
    return [aa_s.index(r) for r in res_str]

def aa_features():
    # Meiler's features
    prop1 = [[1.77, 0.13, 2.43,  1.54,  6.35, 0.17, 0.41],
             [1.31, 0.06, 1.60, -0.04,  5.70, 0.20, 0.28],
             [3.03, 0.11, 2.60,  0.26,  5.60, 0.21, 0.36],
             [2.67, 0.00, 2.72,  0.72,  6.80, 0.13, 0.34],
             [1.28, 0.05, 1.00,  0.31,  6.11, 0.42, 0.23],
             [0.00, 0.00, 0.00,  0.00,  6.07, 0.13, 0.15],
             [1.60, 0.13, 2.95, -0.60,  6.52, 0.21, 0.22],
             [1.60, 0.11, 2.78, -0.77,  2.95, 0.25, 0.20],
             [1.56, 0.15, 3.78, -0.64,  3.09, 0.42, 0.21],
             [1.56, 0.18, 3.95, -0.22,  5.65, 0.36, 0.25],
             [2.99, 0.23, 4.66,  0.13,  7.69, 0.27, 0.30],
             [2.34, 0.29, 6.13, -1.01, 10.74, 0.36, 0.25],
             [1.89, 0.22, 4.77, -0.99,  9.99, 0.32, 0.27],
             [2.35, 0.22, 4.43,  1.23,  5.71, 0.38, 0.32],
             [4.19, 0.19, 4.00,  1.80,  6.04, 0.30, 0.45],
             [2.59, 0.19, 4.00,  1.70,  6.04, 0.39, 0.31],
             [3.67, 0.14, 3.00,  1.22,  6.02, 0.27, 0.49],
             [2.94, 0.29, 5.89,  1.79,  5.67, 0.30, 0.38],
             [2.94, 0.30, 6.47,  0.96,  5.66, 0.25, 0.41],
             [3.21, 0.41, 8.08,  2.25,  5.94, 0.32, 0.42],
             [0.00, 0.00, 0.00,  0.00,  0.00, 0.00, 0.00]]
    return np.array(prop1)
NUM_FEATURES = len(aa_s) + 7

## Model Design

In [164]:
from keras.engine import Model
from keras.layers import Layer, Bidirectional, TimeDistributed, \
    Dense, LSTM, Masking, Input, RepeatVector, Dropout, Convolution1D, \
    BatchNormalization, Activation
from keras.layers.merge import concatenate, add
import keras.backend as K
from keras.regularizers import l2
from keras.callbacks import LearningRateScheduler, EarlyStopping

In [165]:
class MaskingByLambda(Layer):
    def __init__(self, func, **kwargs):
        self.supports_masking = True
        self.mask_func = func
        super(MaskingByLambda, self).__init__(**kwargs)

    def compute_mask(self, input, input_mask=None):
        return self.mask_func(input, input_mask)

    def call(self, x, mask=None):
        exd_mask = K.expand_dims(self.mask_func(x, mask), axis=-1)
        return x * K.cast(exd_mask, K.floatx())

def mask_by_input(tensor):
        return lambda input, mask: tensor


# 1D convolution that supports masking by retaining the mask of the input
class MaskedConvolution1D(Convolution1D):
    def __init__(self, *args, **kwargs):
        self.supports_masking = True
        assert kwargs['padding'] == 'same' # Only makes sense for 'same'
        super(MaskedConvolution1D, self).__init__(*args, **kwargs)

    def compute_mask(self, input, input_mask=None):
        return input_mask

    def call(self, x, mask=None):
        assert mask is not None
        mask = K.expand_dims(mask, axis=-1)
        x = super(MaskedConvolution1D, self).call(x)
        return x * K.cast(mask, K.floatx())

In [166]:
def base_ab_seq_model(max_cdr_len):
    input_ab = Input(shape=(max_cdr_len, NUM_FEATURES))
    label_mask = Input(shape=(max_cdr_len,))

    seq = MaskingByLambda(mask_by_input(label_mask))(input_ab)
    loc_fts = MaskedConvolution1D(28, 3, padding='same', activation='elu',
                                  kernel_regularizer=l2(0.01))(seq)

    res_fts = add([seq, loc_fts])

    glb_fts = Bidirectional(LSTM(256, dropout=0.15, recurrent_dropout=0.2,
                                 return_sequences=True),
                            merge_mode='concat')(res_fts)

    fts = Dropout(0.3)(glb_fts)
    probs = TimeDistributed(Dense(1, activation='sigmoid',
                                  kernel_regularizer=l2(0.01)))(fts)
    return input_ab, label_mask, res_fts, probs

In [167]:
def ab_seq_model(max_cdr_len):
    input_ab, label_mask, _, probs = base_ab_seq_model(max_cdr_len)
    model = Model(inputs=[input_ab, label_mask], outputs=probs)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['binary_accuracy', false_pos, false_neg],
                  sample_weight_mode="temporal")
    return model

In [168]:
def conv_output_ab_seq_model(max_cdr_len):
    input_ab, label_mask, loc_fts, probs = base_ab_seq_model(max_cdr_len)
    model = Model(inputs=[input_ab, label_mask], outputs=[probs, loc_fts])
    return model

In [169]:
def false_neg(y_true, y_pred):
    return K.squeeze(K.clip(y_true - K.round(y_pred), 0.0, 1.0), axis=-1)


def false_pos(y_true, y_pred):
    return K.squeeze(K.clip(K.round(y_pred) - y_true, 0.0, 1.0), axis=-1)


## Evaluation Metrics

In [170]:
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, matthews_corrcoef

In [171]:
def flatten_with_lengths(matrix, lengths):
    seqs = []
    for i, example in enumerate(matrix):
        seq = example[:lengths[i]]
        seqs.append(seq)
    return np.concatenate(seqs)


In [172]:
def youden_j_stat(fpr, tpr, thresholds):
    j_ordered = sorted(zip(tpr - fpr, thresholds))
    return j_ordered[-1][1]


def compute_classifier_metrics(labels, probs):
    matrices = []
    aucs = []
    mcorrs = []
    jstats = []

    for l, p in zip(labels, probs):
        jstats.append(youden_j_stat(*roc_curve(l, p)))

    jstat_scores = np.array(jstats)
    jstat = np.mean(jstat_scores)
    jstat_err = 2 * np.std(jstat_scores)

    threshold = jstat

    print("Youden's J statistic = {} +/- {}. Using it as threshold.".format(jstat, jstat_err))

    for l, p in zip(labels, probs):
        aucs.append(roc_auc_score(l, p))
        l_pred = (p > threshold).astype(int)
        matrices.append(confusion_matrix(l, l_pred))
        mcorrs.append(matthews_corrcoef(l, l_pred))

    matrices = np.stack(matrices)
    mean_conf = np.mean(matrices, axis=0)
    errs_conf = 2 * np.std(matrices, axis=0)

    tps = matrices[:, 1, 1]
    fns = matrices[:, 1, 0]
    fps = matrices[:, 0, 1]

    recalls = tps / (tps + fns)
    precisions = tps / (tps + fps)

    rec = np.mean(recalls)
    rec_err = 2 * np.std(recalls)
    prec = np.mean(precisions)
    prec_err = 2 * np.std(precisions)

    fscores = 2 * precisions * recalls / (precisions + recalls)
    fsc = np.mean(fscores)
    fsc_err = 2 * np.std(fscores)

    auc_scores = np.array(aucs)
    auc = np.mean(auc_scores)
    auc_err = 2 * np.std(auc_scores)

    mcorr_scores = np.array(mcorrs)
    mcorr = np.mean(mcorr_scores)
    mcorr_err = 2 * np.std(mcorr_scores)

    print("Mean confusion matrix and error")
    print(mean_conf)
    print(errs_conf)

    print("Recall = {} +/- {}".format(rec, rec_err))
    print("Precision = {} +/- {}".format(prec, prec_err))
    print("F-score = {} +/- {}".format(fsc, fsc_err))
    print("ROC AUC = {} +/- {}".format(auc, auc_err))
    print("MCC = {} +/- {}".format(mcorr, mcorr_err))


## K-fold Cross validation

In [173]:
from sklearn.model_selection import KFold

In [174]:
def structure_ids_to_selection_mask(idx, num_structures):
    mask = np.zeros((num_structures * 6, ), dtype=np.bool)
    offset = idx * 6
    for i in range(6):
        mask[offset + i] = True
    return mask

In [185]:
def kfold_cv_eval(model_func, output_file,
                  weights_template, seed=0):
#     cdrs, lbls, masks = dataset["cdrs"], dataset["lbls"], dataset["masks"]
    kf = KFold(n_splits=10, random_state=seed, shuffle=True)

    all_lbls = []
    all_probs = []
    all_masks = []

    num_structures = int(len(cdrs) / 6)
    for i, (train_ids, test_ids) in enumerate(kf.split(np.arange(num_structures))):
        print("Fold: ", i + 1)

        train_idx = structure_ids_to_selection_mask(train_ids, num_structures)
        test_idx = structure_ids_to_selection_mask(test_ids, num_structures)

        cdrs_train, lbls_train, mask_train = cdrs[train_idx], lbls[train_idx], masks[train_idx]
        cdrs_test, lbls_test, mask_test = cdrs[test_idx], lbls[test_idx], masks[test_idx]

        example_weight = np.squeeze((lbls_train * 1.5 + 1) * mask_train)
        test_ex_weight = np.squeeze((lbls_test * 1.5 + 1) * mask_test)
        model = ab_seq_model(max_cdr_len)

        rate_schedule = lambda e: 0.001 if e >= 10 else 0.01

        model.fit([cdrs_train, np.squeeze(mask_train)],
                  lbls_train, batch_size=32, epochs=18,
                  # For informational purposes about the best number of epochs
                  # TODO: replace for proper evaluation
                  # validation_data=([cdrs_test, np.squeeze(mask_test)],
                  #                  lbls_test, test_ex_weight),
                  sample_weight=example_weight,
                  callbacks=[LearningRateScheduler(rate_schedule)])

        model.save_weights(weights_template.format(i))

        probs_test = model.predict([cdrs_test, np.squeeze(mask_test)])
        all_lbls.append(lbls_test)
        all_probs.append(probs_test)
        all_masks.append(mask_test)

    lbl_mat = np.concatenate(all_lbls)
    prob_mat = np.concatenate(all_probs)
    mask_mat = np.concatenate(all_masks)

    with open(output_file, "wb") as f:
        pickle.dump((lbl_mat, prob_mat, mask_mat), f)

In [186]:
def run_cv(output_folder, num_iters=10):
#     cache_file = dataset.split("/")[-1] + ".p"
#     dataset = open_dataset(dataset, dataset_cache=cache_file)
    model_factory = lambda: ab_seq_model(Max_len_CDR)

    makedirs(output_folder + "/weights", exist_ok=True)
    iters = range(num_iters) if type(num_iters) is int else range(*num_iters)
    for i in iters:
        print("Crossvalidation run", i+1)
        output_file = "{}/run-{}.p".format(output_folder, i)
        weights_template = output_folder + "/weights/run-" + \
                           str(i) + "-fold-{}.h5"
        kfold_cv_eval(model_factory, 
                      output_file, weights_template, seed=i)

In [187]:
def open_crossval_results(folder, num_results,
                          loop_filter=None, flatten_by_lengths=True):
    class_probabilities = []
    labels = []

    for r in range(num_results):
        result_filename = "{}/run-{}.p".format(folder, r)
        with open(result_filename, "rb") as f:
            lbl_mat, prob_mat, mask_mat = pickle.load(f)

        # Get entries corresponding to the given loop
        if loop_filter is not None:
            lbl_mat = lbl_mat[loop_filter::6]
            prob_mat = prob_mat[loop_filter::6]
            mask_mat = mask_mat[loop_filter::6]

        if not flatten_by_lengths:
            class_probabilities.append(prob_mat)
            labels.append(lbl_mat)
            continue

        # Discard sequence padding
        seq_lens = np.sum(np.squeeze(mask_mat), axis=1)
        p = flatten_with_lengths(prob_mat, seq_lens)
        l = flatten_with_lengths(lbl_mat, seq_lens)

        class_probabilities.append(p)
        labels.append(l)

    return labels, class_probabilities

## Plotting

In [ ]:
def plot_roc_curve(labels_test, probs_test, colours=("#0072CF", "#68ACE5"),
                   label="This method", plot_fig=None):
    if plot_fig is None:
        plot_fig = plt.figure(figsize=(3.7, 3.7), dpi=400)
    ax = plot_fig.gca()

    num_runs = len(labels_test)
    tprs = np.zeros((num_runs, 10000))
    fprs = np.linspace(0.0, 1.0, num=10000)

    for i in range(num_runs):
        l = labels_test[i]
        p = probs_test[i]

        fpr, tpr, _ = metrics.roc_curve(l.flatten(), p.flatten())

        for j, fpr_val in enumerate(fprs):  # Inefficient, but good enough
            for t, f in zip(tpr, fpr):
                if f >= fpr_val:
                    tprs[i, j] = t
                    break

    avg_tpr = np.average(tprs, axis=0)
    err_tpr = np.std(tprs, axis=0)

    ax.plot(fprs, avg_tpr, c=colours[0], label=label)

    btm_err = avg_tpr - 2 * err_tpr
    btm_err[btm_err < 0.0] = 0.0
    top_err = avg_tpr + 2 * err_tpr
    top_err[top_err > 1.0] = 1.0

    ax.fill_between(fprs, btm_err, top_err, facecolor=colours[1])

    ax.set_ylabel("True positive rate")
    ax.set_xlabel("False positive rate")

    ax.legend()

    return plot_fig

In [ ]:
def plot_pr_curve(labels_test, probs_test, colours=("#0072CF", "#68ACE5"),
                  label="This method", plot_fig=None):
    if plot_fig is None:
        plot_fig = plt.figure(figsize=(4.5, 3.5), dpi=300)
    ax = plot_fig.gca()

    num_runs = len(labels_test)
    precs = np.zeros((num_runs, 10000))
    recalls = np.linspace(0.0, 1.0, num=10000)

    for i in range(num_runs):
        l = labels_test[i]
        p = probs_test[i]

        prec, rec, _ = metrics.precision_recall_curve(l.flatten(), p.flatten())

        # Maximum interpolation
        for j in range(len(prec)):
            prec[j] = prec[:(j+1)].max()

        prec = list(reversed(prec))
        rec = list(reversed(rec))

        for j, recall in enumerate(recalls):  # Inefficient, but good enough
            for p, r in zip(prec, rec):
                if r >= recall:
                    precs[i, j] = p
                    break

    avg_prec = np.average(precs, axis=0)
    err_prec = np.std(precs, axis=0)

    ax.plot(recalls, avg_prec, c=colours[0], label=label)

    btm_err = avg_prec - 2 * err_prec
    btm_err[btm_err < 0.0] = 0.0
    top_err = avg_prec + 2 * err_prec
    top_err[top_err > 1.0] = 1.0

    ax.fill_between(recalls, btm_err, top_err, facecolor=colours[1])

    ax.set_ylabel("Precision")
    ax.set_xlabel("Recall")
    ax.legend()

    print("Average precision: {}".format(np.mean(avg_prec)))

    return plot_fig


## Padding extra 2 residues on each side of CDR

In [178]:
def getCDRLoops_padded(seq,chain,_scheme='chothia'):
    #seq is numbered
    if chain=='L':
        pass
    elif chain=='H':
        pass
    else:
        raise ValueError('chain must be "H" or "L"')
    from collections import deque      
# chain = 'H'
# _scheme='chothia'
    toret = {}
    for i in ['1','2','3','4']:
        if i in ['1','2','3']:
            loop = chain + i
            toret[loop] = {'seq':{},'seqstr':'','length':0}
            target = getCDRPos(loop,_scheme)
    #         a = target[0]-2
            for j in [1,1]:
                target = deque(target)
                target.appendleft(str(int(target[0])-j)) ## adding 2 seq to CDR in front
                target.append(str(int(target[-1])+j)) ## adding 2 seq to CDR at back
    #     
    #         print(loop,target,target[-1])
            for n in target:
                if n in seq:
                    if seq[n]!='-':
                        toret[loop]['seq'][n] = seq[n]
                        toret[loop]['length']+=1
                        toret[loop]['seqstr']+=seq[n]

            fw = chain + i + 'F'
            toret[fw] = {'seq':{},'seqstr':'','length':0}
            for n in getCDRPos(fw,_scheme):
                if n in seq:
                    if seq[n]!='-':
                        toret[fw]['seq'][n] = seq[n]
                        toret[fw]['length']+=1
                        toret[fw]['seqstr']+=seq[n]  
    return toret

## Read summary file

In [105]:
df=[]
# with open('20200610_0818458_summary.tsv') as tsvfile:
#     reader = csv.reader(tsvfile, delimiter='\t')
#     for row in reader:
#         df.append(row[0:5])
#     data = pd.DataFrame(df)
#     data.columns = data.iloc[0]
#     sample = data[1:]
#     sample_file = sample.drop_duplicates(['pdb'])
#     sample_file = sample_file[sample_file['antigen_chain'] != 'NA']
sample_file = pd.read_csv("/Users/apawar/Desktop/PERITIA/summary_file_cleaned.csv")
"# sample_file\n",
Hchain = sample_file['Hchain'].to_numpy()
Lchain = sample_file['Lchain'].to_numpy()
pdb = sample_file['pdb'].to_numpy()

### Extract only CDR seq from both heavy and light chain to calculate max length of CDR

In [106]:
def extract_cdr_loops(pdb,pdb_path):
    CDR = []
    Type = []
    pdb_name = []
    seq_no =[]

    for i in range(len(pdb)):
        cdrs = {}
        ppdb = PandasPdb().read_pdb(pdb_path + pdb[i]+'.pdb')
        sequence = ppdb.amino3to1()
        vh = ''.join(sequence[sequence.chain_id== Hchain[i]].residue_name.values)
        vh_numbered, vh_info = numberSequence(vh)
        Chot= getCDRLoops_padded(vh_numbered,'H','chothia')
        cdrs.update(Chot)

        vh1 = ''.join(sequence[sequence.chain_id== Lchain[i]].residue_name.values)
        vh_numbered1, vh_info1 = numberSequence(vh1)
        Chot1= getCDRLoops_padded(vh_numbered1,'L','chothia')
        cdrs.update(Chot1)

        for key, value in cdrs.items():
            if "F" not in key:
                k = key
                Type.append(k)
                seq = cdrs[k]['seqstr']
                res_no = cdrs[k]['seq']
                seq_no.append(list(res_no.keys()))
                CDR.append(seq) 
                pdb_name.append(pdb[i])
    return pdb_name,Type,seq_no, CDR

In [107]:
# Checking the maximum length of CDR loop for padding
def maxi_len(all_variables):
    max_length = 0
    for i in range(len(all_variables)):
        length = len(all_variables[i])
        max_length = max(max_length,length)
    #     max_length = length
    return max_length

In [108]:
def get_variables(dist_matx_path_heavy,dist_matx_path_light,cdr_loop,cut_off):
    """
    For every CDR get the corresponding seq and check if dist is less than cutoff
    and substitute 1 orelse 0 
    
    E.g. - CDR - TCRASGNIHNYLAWY
           Seq.no - ['22','23','24','25','26','27','28','29','30','31','32','33','34','35','36']
           
           Output - [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
    """
    pdb_name = cdr_loop['PDB_Name'].to_numpy()
    seq_no = cdr_loop['seq_no'].to_numpy()
    chain_type = cdr_loop['Type'].to_numpy()

    all_var = []
    for i in range(len(pdb_name)):
        if chain_type[i] == 'H1' or chain_type[i] == 'H2' or chain_type[i] == 'H3':
            dist = pd.read_pickle(dist_matx_path_heavy+pdb_name[i]+"heavy"+".pkl")
            var = []
            for j in range(len(seq_no[i])):
                first_index = dist.index.get_level_values(0)[0] ##get the first index for slicing
                row_slc = dist.loc[(first_index, seq_no[i][j])] ##slice multi index for specific row

                if np.any(row_slc < cut_off) == True:
                    var.append(1)
                else:
                    var.append(0)
            all_var.append(var)

        elif chain_type[i] == 'L1' or chain_type[i] == 'L2' or chain_type[i] == 'L3':
            dist = pd.read_pickle(dist_matx_path_light+pdb_name[i]+"light"+".pkl")
            var = []
            for j in range(len(seq_no[i])):
                first_index = dist.index.get_level_values(0)[0] ##get the first index for slicing
                row_slc = dist.loc[(first_index, seq_no[i][j])] ##slice multi index for specific row

                if np.any(row_slc < cut_off) == True:
                    var.append(1)
                else:
                    var.append(0)
            all_var.append(var)
    
    return all_var

In [109]:
Max_len_CDR = maxi_len(CDR)
Max_len_CDR

38

In [110]:
## Executing extract_cdr_loops and storing everyting in df 
pdb_path = "/Users/apawar/Desktop/PERITIA/cleaned_pdb_shweta/"
pdb_name,Type,seq_no,CDR = extract_cdr_loops(pdb,pdb_path)

### Dataframe of odb,seqno and CDR
CDR_heavy_clean_padded = pd.DataFrame(list(zip(pdb_name,Type,seq_no, CDR)), 
               columns =['PDB_Name','Type','seq_no','CDR'])
CDR_heavy_clean_padded.to_pickle("./CDRs_seqno_pdb.pkl")

## Generating Label

In [219]:
cdr_loop = pd.read_pickle("CDRs_seqno_pdb.pkl")
cdr_loop = cdr_loop.drop(cdr_loop[cdr_loop.PDB_Name.isin(["4ydl","4ydv","5ig7","5ies","4hii","5w0d","5i9q","4hkx",
                                                          "5te4","5te6","4xvs","4xvt","3gbm","2xqb","5if0","4s1q",
                                                          "4xmp","4xny","3mlt","6mvl","4xh2","5ifj","5te7","5occ",
                                                          "4dqo","4lsq","4lsp","4lsr","4yaq","3h3p","3idi",
                                                          "3lrs","3qnz","5x08","5alc","2p8m","6pbw","5csz",
                                                          "6bkb","5ado","4xaw","5wnb","5dt1","4ob5","4xbe",
                                                          "5fgb","4y5y","3u4e","4xcf","4rnr","3drq","6uyf",
                                                          "4m8q","3d0l","2jb6","3u2s","3lhp","3mug"])].index)
cdr_loop.reset_index(drop=True, inplace=True)

dist_matx_path_light = "/Users/apawar/Desktop/PERITIA/Light_chain_dist_clean/"
dist_matx_path_heavy = "/Users/apawar/Desktop/PERITIA/Heavy_chain_dist_clean/"
cut_off = 3
all_variables = get_variables(dist_matx_path_heavy,dist_matx_path_light,cdr_loop,cut_off)

In [211]:
###Matrix of labels

cont_mats = []
"""
Pad the variables to Max CDR len with 0
Here max len of CDR is 38

E.g - Input = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
    Output = [0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[1.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.]

"""
for i in range(len(all_variables)):
    cont_mat = np.array(all_variables[i], dtype=float)
    cont_mat_pad = np.zeros((Max_len_CDR, 1))
    cont_mat_pad[:cont_mat.shape[0], 0] = cont_mat
    cont_mats.append(cont_mat_pad)

lbls = np.stack(cont_mats)
lbls.shape

(3816, 38, 1)

## Generating input X matrix using Meiler's encoding and padding

In [212]:
##cdrs (padding Meiler's encoding of shape)
"""
From the CDR dataframe, extract the CDR column, apply one-hot encoding on every residue,
pad the rest with Meiler's encoding
E.g. - if the length of residue is 10, will get 10 arrays for every residue which will be a combination of both
one-hot encoding and Meiler's encoding. So, for every residue we will have (21+7) features(channels).

Assume we have Max_len of CDR as 21. We will have to pad the remaining 11 with zeros 
21 columns as its the max len of the CDR.
If we have 1998 such CDRs the shape of out input cdrs matrix will be

cdrs.shape = (1998, 21, 28)
"""
cdr_loop = pd.read_pickle("CDRs_seqno_pdb.pkl")
cdr_loop = cdr_loop.drop(cdr_loop[cdr_loop.PDB_Name.isin(["4ydl","4ydv","5ig7","5ies","4hii","5w0d","5i9q","4hkx",
                                                          "5te4","5te6","4xvs","4xvt","3gbm","2xqb","5if0","4s1q",
                                                          "4xmp","4xny","3mlt","6mvl","4xh2","5ifj","5te7","5occ",
                                                          "4dqo","4lsq","4lsp","4lsr","4yaq","3h3p","3idi",
                                                          "3lrs","3qnz","5x08","5alc","2p8m","6pbw","5csz",
                                                          "6bkb","5ado","4xaw","5wnb","5dt1","4ob5","4xbe",
                                                          "5fgb","4y5y","3u4e","4xcf","4rnr","3drq","6uyf",
                                                          "4m8q","3d0l","2jb6","3u2s","3lhp","3mug"])].index)
cdr_loop.reset_index(drop=True, inplace=True)

NUM_FEATURES = 28
cdr_mats = []
for i in range(len(cdr_loop['CDR'])):
#     on_hot = seq_to_one_hot(df['CDR'][i])
    cdr_mat = seq_to_one_hot(cdr_loop['CDR'][i])
    cdr_mat_pad = np.zeros((Max_len_CDR, NUM_FEATURES))
    cdr_mat_pad[:cdr_mat.shape[0], :] = cdr_mat
    cdr_mats.append(cdr_mat_pad)
cdrs = np.stack(cdr_mats)
cdrs.shape

(3816, 38, 28)

In [213]:
##masks (1 for len of chain rest 0)
cdr_masks = []
for i in range(len(cdr_loop['CDR'])):
    cdr_mask = np.zeros((Max_len_CDR, 1), dtype=int)
    cdr_mask[:len(cdr_loop['CDR'][i]), 0] = 1
    cdr_masks.append(cdr_mask)
masks = np.stack(cdr_masks)
masks.shape

(3816, 38, 1)

## Running model with 10-fold cross validation

In [214]:
max_cdr_len = 38

In [220]:
from os import makedirs
output_folder = "Crossvalidation_cut_off_3"
run_cv(output_folder, num_iters=10)

### Compute Metrics for cut_off = 5A

In [221]:
import matplotlib.pyplot as plt
from sklearn import metrics
plt.rcParams["font.family"] = "Arial"

cv_result_folder = "Crossvalidation_cut_off_5"
cv_num_iters = 10

for i, loop in enumerate(["H1", "H2", "H3", "L1", "L2", "L3"]):
    print("Classifier metrics for loop type", loop)
    labels, probs = open_crossval_results(cv_result_folder, 10, i)
    compute_classifier_metrics(labels, probs)
    
    print("Plotting PR curves")
#     labels, probs = open_crossval_results(cv_result_folder, cv_num_iters)
    fig = plot_pr_curve(labels, probs, colours=("#0072CF", "#68ACE5"),
                        label="Parapred",plot_fig = None)
    fig
    fig1 = plot_roc_curve(labels, probs, colours=("#0072CF", "#68ACE5"),
                        label="Parapred",plot_fig = None)
    fig1
# #     fig = plot_pr_curve(labels_abip, probs_abip, colours=("#D6083B", "#EB99A9"),
# #                         label="Parapred using ABiP data", plot_fig=fig)
# fig = plot_abip_pr(fig)
# fig.savefig("pr.eps")

print("Computing classifier metrics")
# compute_classifier_metrics(labels, probs)
    

## Compute Metrics for cut_off = 5A

In [222]:
import matplotlib.pyplot as plt
from sklearn import metrics
plt.rcParams["font.family"] = "Arial"

cv_result_folder = "Crossvalidation_cut_off_3"
cv_num_iters = 10

for i, loop in enumerate(["H1", "H2", "H3", "L1", "L2", "L3"]):
    print("Classifier metrics for loop type", loop)
    labels, probs = open_crossval_results(cv_result_folder, 10, i)
    compute_classifier_metrics(labels, probs)
    
    print("Plotting PR curves")
#     labels, probs = open_crossval_results(cv_result_folder, cv_num_iters)
    fig = plot_pr_curve(labels, probs, colours=("#0072CF", "#68ACE5"),
                        label="Parapred"+loop,plot_fig = None)
    fig
    fig1 = plot_roc_curve(labels, probs, colours=("#0072CF", "#68ACE5"),
                        label="Parapred"+loop,plot_fig = None)
    fig1